In [1]:
import pandas as pd
from os.path import join
data_path = join('..', 'Data')

In [8]:
path = join(data_path, 'october_generator2017.xlsx')
operating = pd.read_excel(path, sheet_name='Operating', header=1, skip_footer=1)
retired = pd.read_excel(path, sheet_name='Retired', header=1, skip_footer=1)
planned = pd.read_excel(path, sheet_name='Planned', header=1, skip_footer=1)

In [9]:
operating.head()

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
0,195,Alabama Power Co,2,Bankhead Dam,Electric Utility,AL,1,NaN,53.9,56,...,,,,,Tuscaloosa,33.458665,-87.35682,Map,Map,SOCO
1,195,Alabama Power Co,3,Barry,Electric Utility,AL,1,NaN,153.1,55,...,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
2,195,Alabama Power Co,3,Barry,Electric Utility,AL,2,NaN,153.1,55,...,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
3,195,Alabama Power Co,3,Barry,Electric Utility,AL,4,NaN,403.7,362,...,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO
4,195,Alabama Power Co,3,Barry,Electric Utility,AL,5,NaN,788.8,738.5,...,,,,,Mobile,31.006900,-88.01030,Map,Map,SOCO


In [12]:
for df in [operating, planned, retired]:
    df.drop(df.index[df['Balancing Authority Code'] != 'ERCO'],
            axis=0, inplace=True)


In [13]:
operating.head()

,Entity ID,Entity Name,Plant ID,Plant Name,Sector,Plant State,Generator ID,Unit Code,Nameplate Capacity (MW),Net Summer Capacity (MW),...,Planned Derate of Summer Capacity (MW),Planned Uprate Year,Planned Uprate Month,Planned Uprate of Summer Capacity (MW),County,Latitude,Longitude,Google Map,Bing Map,Balancing Authority Code
527,54888,NRG Texas Power LLC,298,Limestone,IPP Non-CHP,TX,1,NaN,893.0,831,...,,,,,Limestone,31.421900,-96.25250,Map,Map,ERCO
528,54888,NRG Texas Power LLC,298,Limestone,IPP Non-CHP,TX,2,NaN,956.8,858,...,,,,,Limestone,31.421900,-96.25250,Map,Map,ERCO
1402,7751,Guadalupe Blanco River Authority,791,Canyon,Electric Utility,TX,1,NaN,3.0,3,...,,,,,Comal,29.870560,-98.19473,Map,Map,ERCO
1403,7751,Guadalupe Blanco River Authority,791,Canyon,Electric Utility,TX,2,NaN,3.0,3,...,,,,,Comal,29.870560,-98.19473,Map,Map,ERCO
1414,6958,City of Garland - (TX),794,Lewisville,Electric Utility,TX,1,NaN,2.8,2,...,,,,,Denton,33.068473,-96.96381,Map,Map,ERCO


In [31]:
def capacity_by_year(op, ret, year):
    op_year = op.loc[op['Operating Year'] <= year]
    ret_year = ret.loc[ret['Retirement Year'] >= year]
    
    df = pd.concat([op_year, ret_year])
    df.drop(labels=['Entity ID',
                    'Operating Month',
                    'Operating Year',
                    'Plant ID',
                    'Retirement Month',
                    'Retirement Year'], axis=1, inplace=True)
    
    return df

In [17]:
tech_groups = {
    'natural gas cc' : ['Natural Gas Fired Combined Cycle'],
    'natural gas single' : ['Natural Gas Fired Combustion Turbine',
                             'Natural Gas Internal Combustion Engine'],
    'natural gas steam' : ['Natural Gas Steam Turbine'],
    'coal' : ['Conventional Steam Coal'],
    'nuclear' : ['Nuclear'],
    'wind' : ['Onshore Wind Turbine'],
    'other' : ['All Other',
               'Batteries',
               'Conventional Hydroelectric',
               'Landfill Gas',
               'Other Gases',
               'Other Waste Biomass',
               'Petroleum Coke',
               'Petroleum Liquids',
               'Solar Photovoltaic',
               'Wood/Wood Waste Biomass']
}

In [22]:
for key, value in tech_groups.items():
    operating.loc[operating['Technology'].isin(value), 'tech'] = key
    retired.loc[retired['Technology'].isin(value), 'tech'] = key
    planned.loc[planned['Technology'].isin(value), 'tech'] = key

In [33]:
df = capacity_by_year(operating, retired, 2015).groupby('tech').sum()
df['year'] = 2015
df

,Nameplate Capacity (MW),year
tech,,
coal,20089.6,2015
natural gas cc,41363.0,2015
natural gas single,7635.9,2015
natural gas steam,15567.9,2015
nuclear,5138.6,2015
other,1793.3,2015
wind,16270.9,2015
